<div style="text-align:center">
    <h1>
        VSL RL : SUMO Simulation (DQL)
    </h1>
</div>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.optimizers import Adam
import sumo_env as env
from rl.agents.dqn import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory
import gym
import datetime

# Load the TensorBoard notebook extension
from tensorflow.keras.callbacks import TensorBoard
%load_ext tensorboard


/Users/asmae/miniforge3/envs/dql_vsl/lib/python3.8/site-packages/gym/spaces/box.py:78: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


Initialize the environment

In [2]:
import gym
from gym.envs.registration import register
from SUMOInitializeEnv import SUMOEnv_Initializer
register(
    id='SumoGUI-v0',
    entry_point='SUMOInitializeEnv:SUMOEnv_Initializer'
)
env = gym.make('SumoGUI-v0')
np.random.seed(0)
env.seed(0)
tf.__version__

'2.8.0'

Build model

In [4]:
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dense(env.action_space.size))
model.add(Activation('linear'))
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 2)                 0         
                                                                 
 dense_2 (Dense)             (None, 32)                96        
                                                                 
 activation_2 (Activation)   (None, 32)                0         
                                                                 
 dense_3 (Dense)             (None, 6)                 198       
                                                                 
 activation_3 (Activation)   (None, 6)                 0         
                                                                 
Total params: 294
Trainable params: 294
Non-trainable params: 0
_________________________________________________________________
None


In [5]:
%reload_ext tensorboard
log_dir = "/users/asmae/Desktop/PHD/rl_vsl_sumo/rl_vsl_sumo/gym_sumo/envs/logs/fit/"+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = [TensorBoard(log_dir=log_dir, histogram_freq=1,write_graph=True,
                         write_images=True,
                         update_freq='epoch',
                         profile_batch=2,
                         embeddings_freq=1)]

#### Deep Q Learning

In [6]:
import epsilon_greedy as EpsilonGreedy
#tf.debugging.experimental.enable_dump_debug_info("/users/asmae/Desktop/PHD/TS_LOGDIR", tensor_debug_mode="FULL_HEALTH", circular_buffer_size=-1)
policy = EpsGreedyQPolicy()
memory = SequentialMemory(limit=50000, window_length=1)
dqn = DQNAgent(model=model, nb_actions=env.action_space.size, memory=memory, nb_steps_warmup=30,
target_model_update=1e-2, policy=policy)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

/Users/asmae/miniforge3/envs/dql_vsl/lib/python3.8/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
2022-03-07 23:39:04.648889: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-07 23:39:04.649026: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-03-07 23:39:04.654287: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-03-07 23:39:04.654362: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is ena

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



In [7]:
import traceback
try:
    #tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    dqn.fit(env, nb_steps=5000, visualize=True, verbose=2 
            #,callbacks=tensorboard_callback
            )
    env.close()
    dqn.test(env, nb_episodes=1, nb_max_episode_steps= 4000, visualize=True)
    env.close()
except ValueError:
    print(traceback.format_exc())
    env.close()

Training for 5000 steps ...
 Retrying in 1 seconds


Fontconfig warning: ignoring UTF-8: not a valid region tag
/Users/asmae/miniforge3/envs/dql_vsl/lib/python3.8/site-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
2022-03-07 23:39:27.712128: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-07 23:39:27.722194: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-07 23:39:27.728679: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
/Users/asmae/miniforge3/envs/dql_vsl/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype

q_values To maximize: [0. 0. 0. 0. 0. 0.]
maximizing action
speed chosen = 19
State : [0.0, 0]
reward: nan
q_values To maximize: [0. 0. 0. 0. 0. 0.]
maximizing action
speed chosen = 19
State : [0.0, 0]
reward: nan
q_values To maximize: [0. 0. 0. 0. 0. 0.]
maximizing action
speed chosen = 19
State : [0.0, 0]
reward: nan
q_values To maximize: [0. 0. 0. 0. 0. 0.]
maximizing action
speed chosen = 19
State : [0.0, 0]
reward: nan
q_values To maximize: [0. 0. 0. 0. 0. 0.]
maximizing action
speed chosen = 19
State : [0.0, 0]
reward: nan
q_values To maximize: [0. 0. 0. 0. 0. 0.]
maximizing action
speed chosen = 19
State : [0.0, 0]
reward: nan
q_values To maximize: [0. 0. 0. 0. 0. 0.]
maximizing action
speed chosen = 19
State : [0.0, 0]
reward: nan
q_values To maximize: [0. 0. 0. 0. 0. 0.]
maximizing action
speed chosen = 19
State : [0.0, 0]
reward: nan
q_values To maximize: [0. 0. 0. 0. 0. 0.]
maximizing action
speed chosen = 19
State : [0.0, 0]
reward: nan
q_values To maximize: [0. 0. 0. 0. 0.

/Users/asmae/miniforge3/envs/dql_vsl/lib/python3.8/site-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')
2022-03-07 23:39:28.905812: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-07 23:39:28.974829: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-07 23:39:28.983751: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-07 23:39:28.991775: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-07 23:39:29.117713: I tensorflow/c

q_values To maximize: [        nan -0.0094782   0.01012896  0.0110632   0.01278519  0.01215646]
maximizing action
speed chosen = 19
State : [0.26455026455026454, 0]
reward: 46.76272727272726
random action
speed chosen = 25
State : [0.26455026455026454, 0]
reward: 46.76272727272726
q_values To maximize: [nan nan  0.  0.  0.  0.]
maximizing action
speed chosen = 19
State : [0.1763668430335097, 0]
reward: 45.99090909090909
q_values To maximize: [nan nan nan  0.  0.  0.]
maximizing action
speed chosen = 19
State : [0.26455026455026454, 0]
reward: 45.804545454545455
q_values To maximize: [nan nan nan  0.  0.  0.]
maximizing action
speed chosen = 19
State : [0.26455026455026454, 0]
reward: 45.804545454545455
q_values To maximize: [nan nan nan  0.  0.  0.]
maximizing action
speed chosen = 19
State : [0.3527336860670194, 0]
reward: 47.740833333333335
random action
speed chosen = 33
State : [0.08818342151675485, 0]
reward: 47.740833333333335
q_values To maximize: [nan nan nan  0.  0.  0.]
maxim

Fontconfig warning: ignoring UTF-8: not a valid region tag


speed chosen = 19
State : [0.0, 0]
reward: nan
speed chosen = 19
State : [0.0, 0]
reward: nan
speed chosen = 19
State : [0.0, 0]
reward: nan
speed chosen = 19
State : [0.0, 0]
reward: 49.29
speed chosen = 19
State : [0.0, 0]
reward: 49.29
speed chosen = 19
State : [0.0, 0]
reward: 49.29
speed chosen = 19
State : [0.0, 0]
reward: 49.29
speed chosen = 19
State : [0.1763668430335097, 0]
reward: 49.29
speed chosen = 19
State : [0.0, 0]
reward: 30.415
speed chosen = 19
State : [0.0, 0]
reward: 30.415
speed chosen = 19
State : [0.1763668430335097, 0]
reward: 30.415
speed chosen = 19
State : [0.0, 0]
reward: 26.22
speed chosen = 19
State : [0.0, 0]
reward: 21.79
speed chosen = 19
State : [0.08818342151675485, 0]
reward: 21.79
speed chosen = 19
State : [0.08818342151675485, 0]
reward: 21.79
speed chosen = 19
State : [0.0, 0]
reward: 31.38555555555556
speed chosen = 19
State : [0.08818342151675485, 0]
reward: 31.38555555555556
speed chosen = 19
State : [0.08818342151675485, 0]
reward: 28.995454

In [11]:
%tensorboard --logdir={log_folder}